### NiN网络

LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。而网络中的网络（NiN）它提出了另外一个思路，即串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络。

#### NiN块

我们知道，卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维。而1×1卷积层可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用1×1卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。NiN同AlexNet和VGG等网络在结构上的主要区别。


<img src="img/nin.svg">

        左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的1×1卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。

#### 知识点

#### 1x1卷积核心
<img src="img/full_padding_no_strides_transposed_small.gif">
    
    因为使用了最小窗口，1×1卷积失去了卷积层可以识别高和宽维度上相邻元素构成的模式的功能。实际上，1×1卷积的主要计算发生在通道维上。
    注意，输入和输出具有相同的高和宽。输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么1×1卷积层的作用与全连接层等价。
    1×1卷积层通常用来调整网络层之间的通道数，并控制模型复杂度

#### 全局平均池化

    CNN网络中常见结构是：卷积、池化和激活。卷积层是CNN网络的核心，激活函数帮助网络获得非线性特征，而池化的作用则体现在降采样：保留显著特征、降低特征维度，增大kernel的感受野。增大感受野能够看到全局信息，比如盲人摸象，看到全局才能更好的知道这是大象
    
全连接与平均池化层

    全连接存在的问题：参数量过大，降低了训练的速度，且很容易过拟合
<img src = "img/20180512165136102.png">
    
    全连接层将卷积层展开成向量之后不还是要针对每个feature map进行分类，而GAP的思路就是将上述两个过程合二为一，一起做了
    对整个网路在结构上做正则化防止过拟合。
    global average pooling 与 average pooling 的差别就在 "global" 这一个字眼上。global 与 local 在字面上都是用来形容 pooling 窗口区域的。 local 是取 feature map 的一个子区域求平均值，然后滑动这个子区域； global 显然就是对整个 feature map 求平均值了。
    
    说白了，“global pooling”就是pooling的 滑窗size 和整张feature map的size一样大。这样，每个 W×H×C 的feature map输入就会被转化为 1×1×C 输出。因此，其实也等同于每个位置权重都为 1/(W×H)的FC层操作。
    等同于输入一个tensor，输出一根vector。

    “global pooling”在滑窗内的具体pooling方法可以是任意的，所以就会被细分为“global avg pooling”、“global max pooling”等。
    
    用一个GAP将N个feature map降维成1*N大小的feature map,再用class个1*1卷积核将1*N的feature map卷成1*class的向量。因此，整个过程在维度上来看相当于一层FC，但是需要注意的是，在使用GAP后，网络收敛速度将会变慢。
    

#### NiN模型

NiN是在AlexNet问世不久后提出的。它们的卷积层设定有类似之处。NiN使用卷积窗口形状分别为11×11、5×5和3×3的卷积层，相应的输出通道数也与AlexNet中的一致。每个NiN块后接一个步幅为2、窗口形状为3×3的最大池化层。

除使用NiN块以外，NiN还有一个设计与AlexNet显著不同：NiN去掉了AlexNet最后的3个全连接层，取而代之地，NiN使用了输出通道数等于标签类别数的NiN块，然后使用全局平均池化层对每个通道中所有元素求平均并直接用于分类。这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层。NiN的这个设计的好处是可以显著减小模型参数尺寸，从而缓解过拟合。然而，该设计有时会造成获得有效模型的训练时间的增加。

#### NiN实现

In [51]:
%%time
# 定义模型  采用函数式接口
# 函数式接口最大的特点就是，网络层layer，甚至模型都可以被调用。

# 实现
#  basic package
import numpy as np
import keras
from keras.optimizers import SGD
from keras.models import Model
from keras.layers import Input, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPool2D, BatchNormalization

# 首先是nin块
def nin_block(x, num_channels, kernel_size, stride, padding):
    x = Conv2D(filters=num_channels, kernel_size=(kernel_size, kernel_size), 
               strides=(stride,stride), padding = padding, activation='relu')(x)
    
    x = Conv2D(filters=num_channels, kernel_size=(1,1), padding='same', activation='relu')(x)
    x = Conv2D(filters=num_channels, kernel_size=(1,1), padding='same', activation='relu')(x)
    return x

# 1、input
img_input = Input(shape=(224, 224, 1))

x = nin_block(img_input, 96, 11, 4, 'valid')

x = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='valid')(x)

x = nin_block(x, 256, 5, 1, 'same')
x = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='valid')(x)

x = nin_block(x, 384, 3, 1, 'same')
x = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='valid')(x)

# 标签类别数是10
x = nin_block(x, 10, 3, 1, 'same')

# 全局平均池化层将窗口形状自动设置成输入的高和宽，实际上就是将feature_map 变成一个值
x = GlobalAveragePooling2D()(x)

model = Model(inputs=img_input, outputs=x)


# compile
sgd = SGD(lr=0.05)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


# sequential1 output shape:        (1, 96, 54, 54)
# pool0 output shape:      (1, 96, 26, 26)
# sequential2 output shape:        (1, 256, 26, 26)
# pool1 output shape:      (1, 256, 12, 12)
# sequential3 output shape:        (1, 384, 12, 12)
# pool2 output shape:      (1, 384, 5, 5)
# dropout0 output shape:   (1, 384, 5, 5)
# sequential4 output shape:        (1, 10, 5, 5)
# pool3 output shape:      (1, 10, 1, 1)
# flatten0 output shape:   (1, 10)

Wall time: 202 ms


In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 224, 224, 1)       0         
_________________________________________________________________
conv2d_192 (Conv2D)          (None, 54, 54, 96)        11712     
_________________________________________________________________
conv2d_193 (Conv2D)          (None, 54, 54, 96)        9312      
_________________________________________________________________
conv2d_194 (Conv2D)          (None, 54, 54, 96)        9312      
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_195 (Conv2D)          (None, 26, 26, 256)       614656    
_________________________________________________________________
conv2d_196 (Conv2D)          (None, 26, 26, 256)       65792     
__________

### GoogLeNet

#### Inception块

<img src="img/inception.svg">

    GoogLeNet中的基础卷积块叫作Inception块。
    Inception块里有4条并行的线路。前3条线路使用窗口大小分别是1×1、3×3和5×5的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做1×1卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用3×3最大池化层，后接1×1卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。

#### GoogLeNet模型

    GoogLeNet跟VGG一样，在主体卷积部分中使用5个模块（block），每个模块之间使用步幅为2的3×3最大池化层来减小输出高宽。第一模块使用一个64通道的7×7卷积层。

    第二模块使用2个卷积层：首先是64通道的1×1卷积层，然后是将通道增大3倍的3×3卷积层。它对应Inception块中的第二条线路。
    
    第三模块串联2个完整的Inception块。第一个Inception块的输出通道数为64+128+32+32=256，其中4条线路的输出通道数比例为64:128:32:32=2:4:1:1。其中第二、第三条线路先分别将输入通道数减小至96/192=1/2和16/192=1/12后，再接上第二层卷积层。第二个Inception块输出通道数增至128+192+96+64=480，每条线路的输出通道数之比为128:192:96:64=4:6:3:2。其中第二、第三条线路先分别将输入通道数减小至128/256=1/2和32/256=1/8。
    
    第四模块更加复杂。它串联了5个Inception块，其输出通道数分别是192+208+48+64=512、160+224+64+64=512、128+256+64+64=512、112+288+64+64=528和256+320+128+128=832。这些线路的通道数分配和第三模块中的类似，首先含3×3卷积层的第二条线路输出最多通道，其次是仅含1×1卷积层的第一条线路，之后是含5×5卷积层的第三条线路和含3×3最大池化层的第四条线路。其中第二、第三条线路都会先按比例减小通道数。这些比例在各个Inception块中都略有不同。
    
    第五模块有输出通道数为256+320+128+128=832和384+384+128+128=1024的两个Inception块。其中每条线路的通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。需要注意的是，第五模块的后面紧跟输出层，该模块同NiN一样使用全局平均池化层来将每个通道的高和宽变成1。最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。


    Inception块相当于一个有4条线路的子网络。它通过不同窗口形状的卷积层和最大池化层来并行抽取信息，并使用1×1卷积层减少通道数从而降低模型复杂度。
    GoogLeNet将多个设计精细的Inception块和其他层串联起来。其中Inception块的通道数分配之比是在ImageNet数据集上通过大量的实验得来的。
    GoogLeNet和它的后继者们一度是ImageNet上最高效的模型之一：在类似的测试精度下，它们的计算复杂度往往更低。


#### googLeNet实现

In [43]:
from keras.models import Model
from keras.layers import Input,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,concatenate, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D,AveragePooling2D
import numpy as np

In [47]:
%%time
# Inception块
def inception(x, c1, c2, c3, c4):
    """
        c1 c2 c3 c4 表示每一个部分的通道数
    """
    br_1x1 = Conv2D(filters=c1, kernel_size=(1, 1), 
                    strides=(1,1), padding = 'same', activation='relu')(x)
    
    br_1x1_and_3x3 = Conv2D(filters=c2[0], kernel_size=(1, 1), 
                            strides=(1,1), padding = 'same', activation='relu')(x)
    br_1x1_and_3x3 = Conv2D(filters=c2[1], kernel_size=(3, 3), 
                            strides=(1,1), padding = 'same', activation='relu')(br_1x1_and_3x3)
    
    br_1x1_and_5x5 = Conv2D(filters=c3[0], kernel_size=(1, 1), 
                            strides=(1,1), padding = 'same', activation='relu')(x)
    br_1x1_and_5x5 = Conv2D(filters=c3[1], kernel_size=(5, 5), 
                            strides=(1,1), padding = 'same', activation='relu')(br_1x1_and_5x5)
    
    br_3x3_and_1x1 = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)
    br_3x3_and_1x1 = Conv2D(filters=c4, kernel_size=(1, 1), 
                            strides=(1,1), padding = 'same', activation='relu')(br_3x3_and_1x1)
    
    # concatenate 就是通道数相加  c1 + c2[1] + c3[1] + c4
    return concatenate([br_1x1, br_1x1_and_3x3, br_1x1_and_5x5, br_3x3_and_1x1], axis=3)


# 1、input  原来是224，现在换成96
img_input = Input(shape=(96, 96, 1))

# 第一个模块 
x = Conv2D(filters = 64, kernel_size = (7,7), strides=(2,2), padding='same', activation='relu')(img_input)
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

# 第二个模块
x = Conv2D(filters = 64, kernel_size = (1,1), strides=(1,1), padding='same', activation='relu')(x)
x = Conv2D(filters = 192, kernel_size = (3,3), strides=(1,1), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

# 第三个模块
x = inception(x, 64, (96, 128), (16, 32), 32)
x = inception(x, 128, (128, 192), (32, 96), 64)
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

# 第4个模块
x = inception(x, 192, (96, 208), (16, 48), 64)
x = inception(x, 160, (112, 224), (24, 64), 64)
x = inception(x, 128, (128, 256), (24, 64), 64)
x = inception(x, 112, (144, 288), (32, 64), 64)
x = inception(x, 256, (160, 320), (32, 128), 128)
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

# 第5个模块
x = inception(x, 256, (160, 320), (32, 128), 128)
x = inception(x, 384, (192, 384), (48, 128), 128)
x = GlobalAveragePooling2D()(x)

# 最后全连接成 class 类别 10
x = Dense(10,activation='softmax')(x)

model = Model(inputs=img_input, outputs=x)


# compile
sgd = SGD(lr=0.05)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


Wall time: 877 ms


In [48]:
lr, num_epochs, batch_size= 0.1, 5, 128

In [49]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_135 (Conv2D)             (None, 48, 48, 64)   3200        input_20[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_26 (MaxPooling2D) (None, 24, 24, 64)   0           conv2d_135[0][0]                 
__________________________________________________________________________________________________
conv2d_136 (Conv2D)             (None, 24, 24, 64)   4160        max_pooling2d_26[0][0]           
__________________________________________________________________________________________________
conv2d_137

#### 总结

    Inception块相当于一个有4条线路的子网络。它通过不同窗口形状的卷积层和最大池化层来并行抽取信息，并使用1×1卷积层减少通道数从而降低模型复杂度。
    GoogLeNet将多个设计精细的Inception块和其他层串联起来。其中Inception块的通道数分配之比是在ImageNet数据集上通过大量的实验得来的。
    GoogLeNet和它的后继者们一度是ImageNet上最高效的模型之一：在类似的测试精度下，它们的计算复杂度往往更低。
